In [ ]:
import tensorflow as tf

# force gpu use
gpus = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_visible_devices(gpus[0], 'GPU')

In [ ]:
import os

from modules.pinn import PINNModel, model1, optm1
from modules.data_gen import DataGenerator, real_u1, get_data

In [ ]:
session = 3
data = get_data(session)
EPOCHS = data['epochs']
koefs = data['big_range']
if not data["session_exists"]:
    os.makedirs(f"../models/s{session}")

In [4]:
model = PINNModel(
    model1(), optm1(), lossf="mae", initial_weights=DataGenerator.init_name(session))

# if there are no initial weights, save current as initial
if not os.path.exists(DataGenerator.init_name(session)):
    model.save(DataGenerator.init_name(session))

dg = DataGenerator(data['x'], data['y'], model.predict, real_u1)

In [ ]:
area_grid_size = (50, 50)
border_grid_size = (50, 50)
dataset = dg.inner_pairs(area_grid_size), dg.border_pairs(border_grid_size)

for koef in koefs:
    if not os.path.exists(dg.name(koef, session)):
        model = PINNModel(
            model1(), optm1(), "mae", initial_weights=DataGenerator.init_name(session))
        # model.reset()
        model.fit(koef, *dataset, EPOCHS)
        model.save(dg.name(koef, session))

In [ ]:
import shutil

shutil.make_archive(f"../models/s{session}", 'zip', f"../models/s{session}")